In [173]:
import os
import pandas as pd
import subprocess
from subprocess import call
import rasterio
import xarray as xr
import glob
from datetime import datetime, timedelta
import warnings

In [174]:
def make_dataframe(mypath):
    hdf_name = []
    for image in os.listdir(mypath):
        if image.endswith('hdf'):
            hdf_name.append(image)
    dates = []
    for name in hdf_name:
        dates.append(name.split('.')[1])
    df_images = pd.DataFrame(list(zip(hdf_name, dates)), columns = ['image', 'date']) 
    grouped = df_images.groupby('date')['image'].apply(list).reset_index()
    grouped['image'].count()
    return grouped

In [175]:
def mosaic_and_clip_lst(date,paths,timeofday):
    os.chdir('/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/lst_8day/composites_2020_2021')
    st_name = date+'_'+timeofday+'_merge.tif'
    print(st_name)
    merge_command = ['gdal_merge.py', "-o",st_name, ' '.join(paths)]
    merge_command = ' '.join(merge_command)
    #cmd_a = 'gdal_merge.py -o '+ st_name'-ul_lr 19.46 34.44 28.29 41.9 '
    subprocess.call(merge_command,shell=True)
    warp_command = ['gdalwarp','-t_srs','EPSG:4326','-te 19.46 34.44 28.29 41.9', st_name,st_name.split('.')[0]+'_warp.tif']
    warp_command = ' '.join(warp_command)
    subprocess.call(warp_command,shell=True)
    call(["rm", st_name])

In [176]:
def create_lst_day(year,grouped):
    for index,row in grouped.iterrows():#['image'].loc[0]:
        paths = []
        for d in row['image']: 
            path = 'HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/lst_8day/'+str(year)+'/'+d+'":MODIS_Grid_8Day_1km_LST:LST_Day_1km'
            paths.append(path)
            #mosaic_and_clip_lst(path[0],paths,timeofday)
        date = d.split('.')[1]
        timeofday = 'day'
        mosaic_and_clip_lst(date,paths,timeofday)

In [177]:
def create_lst_night(year,grouped):
    for index,row in grouped.iterrows():#['image'].loc[0]:
        paths = []
        for d in row['image']: 
            path = 'HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/lst_8day/'+str(year)+'/'+d+'":MODIS_Grid_8Day_1km_LST:LST_Night_1km'
            paths.append(path)
            #mosaic_and_clip_lst(path[0],paths,timeofday)
        date = d.split('.')[1]
        timeofday = 'night'
        mosaic_and_clip_lst(date,paths,timeofday)

In [178]:
years =[2020,2021]
#[2010,2011,2012,2013,2014,2015,2016,2017,2018]

In [179]:
for year in years:
    mypath = "/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/lst_8day/"+str(year)
    os.chdir(mypath)
    grouped = make_dataframe(mypath)
    create_lst_day(year,grouped)
    create_lst_night(year,grouped)

A2020081_day_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2020081_day_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020089_day_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2020089_day_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020097_day_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2020097_day_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020105_day_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2020105_day_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020113_day_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - do

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2020169_night_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020177_night_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2020177_night_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020185_night_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2020185_night_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020193_night_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2020193_night_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020201_night_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Cre

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2021273_day_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2021097_night_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2021097_night_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2021105_night_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2021105_night_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2021113_night_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1084P x 916L.
Processing A2021113_night_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2021121_night_merge.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Creat

In [23]:
grouped.loc[2]['image']

,date,image
0,A2021097,[MYD11A2.A2021097.h19v05.006.2021107094824.hdf...
1,A2021105,[MYD11A2.A2021105.h20v05.006.2021114053150.hdf...
2,A2021113,[MYD11A2.A2021113.h19v04.006.2021122082757.hdf...
3,A2021121,[MYD11A2.A2021121.h20v05.006.2021131025710.hdf...
4,A2021129,[MYD11A2.A2021129.h19v04.006.2021138051028.hdf...
...,...,...
104,A2021269,[MOD11A1.A2021269.h20v05.006.2021278165252.hdf...
105,A2021270,[MOD11A1.A2021270.h20v05.006.2021278181352.hdf...
106,A2021271,[MOD11A1.A2021271.h20v05.006.2021278183332.hdf...
107,A2021272,[MOD11A1.A2021272.h20v05.006.2021279225524.hdf...


# NDVI

In [ ]:
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi

In [36]:
def make_dataframe(mypath):
    hdf_name = []
    for image in os.listdir(mypath):
        if image.endswith('hdf'):
            hdf_name.append(image)
    dates = []
    for name in hdf_name:
        dates.append(name.split('.')[1])
    df_images = pd.DataFrame(list(zip(hdf_name, dates)), columns = ['image', 'date']) 
    grouped = df_images.groupby('date')['image'].apply(list).reset_index()
    grouped['image'].count()
    return grouped

In [72]:
def mosaic_and_clip_ndvi(date,paths,mode=''):
    os.chdir('/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_composites')
    st_name = date+'_'+mode+'_merge.tif'
    print(st_name)
    merge_command = ['gdal_merge.py', "-o",st_name, ' '.join(paths)]
    merge_command = ' '.join(merge_command)
    print(merge_command)
    #cmd_a = 'gdal_merge.py -o '+ st_name'-ul_lr 19.46 34.44 28.29 41.9 '
    subprocess.call(merge_command,shell=True)
    warp_command = ['gdalwarp','-t_srs','EPSG:4326','-te 19.46 34.44 28.29 41.9', st_name,st_name.split('.')[0]+'_warp.tif']
    warp_command = ' '.join(warp_command)
    subprocess.call(warp_command,shell=True)
    call(["rm", st_name])

In [73]:
def create_ndvi_evi(year,grouped):
    for index,row in grouped.iterrows():#['image'].loc[0]:
        paths_ndvi = []
        paths_evi = []
        for d in row['image']: 
            path_ndvi = 'HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/'+ d +'":MODIS_Grid_16DAY_500m_VI:"500m 16 days NDVI"'
            path_evi = 'HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/'+ d +'":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI"'
            paths_ndvi.append(path_ndvi)
            paths_evi.append(path_evi)
            #mosaic_and_clip_lst(path[0],paths,timeofday)
        date = d.split('.')[1]
        timeofday = 'day'
        mosaic_and_clip_ndvi(date,paths_ndvi,'ndvi')
        mosaic_and_clip_ndvi(date,paths_evi,'evi')

In [74]:
for year in years:
    mypath = "/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi"
    os.chdir(mypath)
    grouped = make_dataframe(mypath)
    create_ndvi_evi(year,grouped)
    #create_lst_night(year,grouped

A2020097_ndvi_merge.tif
gdal_merge.py -o A2020097_ndvi_merge.tif HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2020097.h19v04.006.2020114012305.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days NDVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2020097.h20v05.006.2020114012517.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days NDVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2020097.h19v05.006.2020114011403.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days NDVI"
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2168P x 1832L.
Processing A2020097_ndvi_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020097_evi_merge.tif
gdal_merge.py -o A2020097_evi_merge.tif HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A20

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2168P x 1832L.
Processing A2020137_ndvi_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020137_evi_merge.tif
gdal_merge.py -o A2020137_evi_merge.tif HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MYD13A1.A2020137.h19v04.006.2020154001203.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MYD13A1.A2020137.h20v05.006.2020154001216.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MYD13A1.A2020137.h19v05.006.2020154000617.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI"
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2168P x 1832L.
Processing A2020137_evi_merge.tif [1/1] : 0...10...20...30...40...50

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2168P x 1832L.
Processing A2020177_ndvi_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020177_evi_merge.tif
gdal_merge.py -o A2020177_evi_merge.tif HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2020177.h19v05.006.2020196233452.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2020177.h19v04.006.2020196234141.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2020177.h20v05.006.2020196233128.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI"
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2168P x 1832L.
Processing A2020177_evi_merge.tif [1/1] : 0...10...20...30...40...50

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2168P x 1832L.
Processing A2020217_ndvi_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020217_evi_merge.tif
gdal_merge.py -o A2020217_evi_merge.tif HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MYD13A1.A2020217.h20v05.006.2020323151553.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MYD13A1.A2020217.h19v05.006.2020323151122.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MYD13A1.A2020217.h19v04.006.2020323151102.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI"
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2168P x 1832L.
Processing A2020217_evi_merge.tif [1/1] : 0...10...20...30...40...50

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2168P x 1832L.
Processing A2020257_ndvi_merge.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
A2020257_evi_merge.tif
gdal_merge.py -o A2020257_evi_merge.tif HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2020257.h20v05.006.2020280043844.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2020257.h19v04.006.2020280044358.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2020257.h19v05.006.2020280044408.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI"
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2168P x 1832L.
Processing A2020257_evi_merge.tif [1/1] : 0...10...20...30...40...50

...

ERROR 1: Output dataset A2022265_evi_merge_warp.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.



0...10...20...30...40...50...60...70...80...90...100 - done.
A2022273_evi_merge.tif
gdal_merge.py -o A2022273_evi_merge.tif HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2022273.h20v05.006.2022297221650.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2022273.h19v05.006.2022297221353.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI" HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/MOD13A1.A2022273.h19v04.006.2022297221424.hdf":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI"


ERROR 1: Output dataset A2022273_ndvi_merge_warp.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.



0...10...20...30...40...50...60...70...80...90...100 - done.


ERROR 1: Output dataset A2022273_evi_merge_warp.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.



# TIF to nc

In [119]:
veg_path = '/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_composites'
lst_path = '/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/lst_8day/composites_2020_2021'

In [117]:
veg_dest_path = '/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg'

In [135]:
lst_dest_path = '/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst'

In [122]:
evi_files = glob.glob(os.path.join(veg_path, 'A*_evi_merge_warp.tif'))
ndvi_files = glob.glob(os.path.join(veg_path, 'A*_ndvi_merge_warp.tif'))
lst_day_files = glob.glob(os.path.join(lst_path, 'A*_day_merge_warp.tif'))
lst_night_files = glob.glob(os.path.join(lst_path, 'A*_night_merge_warp.tif'))

# ndvi

In [182]:
data_dict = {}
for evi_file in evi_files:
    date_str = os.path.basename(evi_file)[1:8]
    if date_str in data_dict:
        data_dict[date_str].append((evi_file, None))
    else:
        data_dict[date_str] = [(evi_file, None)]
        
for ndvi_file in ndvi_files:
    date_str = os.path.basename(ndvi_file)[1:8]
    if date_str in data_dict:
        for i, (evi_file, ndvi_file_) in enumerate(data_dict[date_str]):
            if ndvi_file_ is None:
                data_dict[date_str][i] = (evi_file, ndvi_file)
                break

In [183]:
for date_str, file_pairs in data_dict.items():
    doy = date_str[4:]
    year = int(date_str[:4])
    date = datetime(year, 1, 1) + timedelta(int(doy) - 1)
    yyyymmdd = date.strftime('%Y%m%d')
    nc_file_name = os.path.join(veg_dest_path, f'{yyyymmdd}_veg.nc')
    print(nc_file_name)
    datasets = []
    for evi_file, ndvi_file in file_pairs:
        if ndvi_file is None:
            print(f"No matching NDVI file found for EVI file {evi_file}")
            continue
        evi_da = xr.open_rasterio(evi_file).squeeze()
        ndvi_da = xr.open_rasterio(ndvi_file).squeeze()
        evi_da.name = 'evi'
        ndvi_da.name = 'ndvi'
        ds = xr.merge([evi_da, ndvi_da])
        datasets.append(ds)
    if datasets:
        combined_ds = xr.concat(datasets, dim='band')
        combined_ds = combined_ds.squeeze()
        combined_ds = combined_ds.drop('band')
        combined_ds = combined_ds.assign_coords(time=yyyymmdd)
        combined_ds = combined_ds.expand_dims('time')
        for attr_name in combined_ds.attrs:
            del ds.attrs[attr_name]
        combined_ds.to_netcdf(nc_file_name)

/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg/20210813_veg.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg/20200711_veg.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg/20210712_veg.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg/20220626_veg.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg/20200601_veg.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg/20210728_veg.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg/20200812_veg.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg/20210602_veg.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg/20220728_veg.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg/202208

# lst

In [180]:
data_lst_dict = {}
for day_file in lst_day_files:
    date_str = os.path.basename(day_file)[1:8]
    if date_str in data_lst_dict:
        data_lst_dict[date_str].append((day_file, None))
    else:
        data_lst_dict[date_str] = [(day_file, None)]

for night_file in lst_night_files:
    date_str = os.path.basename(night_file)[1:8]
    if date_str in data_lst_dict:
        for i, (day_file, night_file_) in enumerate(data_lst_dict[date_str]):
            if night_file_ is None:
                data_lst_dict[date_str][i] = (day_file, night_file)
                break

In [142]:
warnings.filterwarnings("ignore", message="open_rasterio is Deprecated in favor of rioxarray")

In [181]:
for date_str, file_pairs in data_lst_dict.items():
    datasets = []
    doy = date_str[4:]
    year = int(date_str[:4])
    date = datetime(year, 1, 1) + timedelta(int(doy) - 1)
    yyyymmdd = date.strftime('%Y%m%d')
    nc_file_name = os.path.join(lst_dest_path, f'{yyyymmdd}_lst.nc')
    print(nc_file_name)
    for lst_day, lst_night in file_pairs:
        if lst_night is None:
            print(f"No matching LST file found for night file {lst_night}")
            continue
        day_da = xr.open_rasterio(lst_day).squeeze()
        night_da = xr.open_rasterio(lst_night).squeeze()
        day_da.name = 'day'
        night_da.name = 'night'
        ds = xr.merge([day_da, night_da])
        datasets.append(ds)
    if datasets:
        combined_ds = xr.concat(datasets, dim='band')
        combined_ds = combined_ds.squeeze()
        combined_ds = combined_ds.drop('band')
        combined_ds = combined_ds.assign_coords(time=yyyymmdd)
        combined_ds = combined_ds.expand_dims('time')
        for attr_name in combined_ds.attrs:
            del ds.attrs[attr_name]
        combined_ds.to_netcdf(nc_file_name)

/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst/20200820_lst.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst/20200609_lst.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst/20210720_lst.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst/20200617_lst.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst/20200929_lst.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst/20200921_lst.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst/20200524_lst.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst/20200913_lst.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst/20200516_lst.nc
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst/202008